## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [2]:
import os
import pytz
import glob
import pathlib

this_dir = pathlib.Path(os.path.abspath(""))
data_dir = this_dir / "data"

In [3]:
import requests
import pandas as pd
import json
from datetime import datetime
from slugify import slugify

# Download

Retrieve the page

In [4]:
url = "https://services5.arcgis.com/AF1vPy2RauA2oFyC/ArcGIS/rest/services/County_Areas/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=*&returnGeometry=true&returnCentroid=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson&token="

In [5]:
r = requests.get(url)

In [6]:
data = r.json()

# Parse

Parse the result

In [7]:
dict_list = []

In [8]:
for item in data["features"]:
    d = dict(
        county="Santa Barbara",
        area=item["attributes"]["NAME"],
        confirmed_cases=item["attributes"]["TOTAL"],
    )
    dict_list.append(d)

Convert to a dataframe

In [9]:
df = pd.DataFrame(dict_list)

Reformat and set the date

In [10]:
date_url = "https://services5.arcgis.com/AF1vPy2RauA2oFyC/ArcGIS/rest/services/County_Areas/FeatureServer/0?f=json"
date_r = requests.get(date_url)
date_data = date_r.json()

In [11]:
timestamp = date_data["editingInfo"]["lastEditDate"]

In [12]:
timestamp = datetime.fromtimestamp((timestamp / 1000))

In [13]:
latest_date = pd.to_datetime(timestamp).date()

In [14]:
df["county_date"] = latest_date

Clean

In [15]:
df["area"] = df.area.apply(
    lambda x: x.replace("City of", "")
    .replace("Community of", "")
    .replace("Unincorporated Area of", "")
    .replace("Unincorporated Areas", "")
    .replace("Unincorporated Area", "")
)

# Vet

In [16]:
try:
    assert not len(df) > 11
except AssertionError:
    raise AssertionError("Santa Barbara County's area scraper has extra rows")

In [17]:
try:
    assert not len(df) < 11
except AssertionError:
    raise AssertionError("Santa Barbara County's area scraper is missing rows")

## Export

Set date

In [18]:
tz = pytz.timezone("America/Los_Angeles")

In [19]:
today = datetime.now(tz).date()

In [20]:
slug = "santa-barbara"

In [21]:
df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [22]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [23]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [24]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [25]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)